# Install required services

In [ ]:
!pip install geopy

# Import required packages

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Ignore ssl certeficate error

In [ ]:
import ssl
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

# Read data from the Wikipedia page

In [ ]:
import pandas as pd

dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_national_capitals',header=0)
for df in dfs:
    print(df)

In [ ]:
df

Let us check how mant capital cities of world countries are there 

In [ ]:
df.shape

In [ ]:
df.info()

# Data Preparation 

Delete Notes column

In [ ]:
df = df.drop('Notes', axis=1)

Rename Columns of city and country according to standard naming conventions

In [ ]:
df.rename(columns={'City/Town':'City','Country/Territory':'Country'}, inplace=True)

Display the capitals of the World Countries 

In [ ]:
df

In [ ]:
df.info()

Removing parentheses and all data within using Pandas 


In [ ]:
df['City'] = df['City'].str.replace(r"\(.*?\)","")

In [ ]:
df

Adding the lat and long coordinate columns to the data frame 

In [ ]:
df["lat"] = ""
df["lng"] = ""

In [ ]:
df

Check the operation of geop: Use geopy library to get the latitude and longitude values of Abu Dhabi (the Capital of United Arab Emirates).

In [ ]:
address = df ['City'].values[2]
geolocator = Nominatim(user_agent="World_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Abu Dhabi are {}, {}.'.format(latitude, longitude))

Obtaining the coordinates data of the World countries’ Capitals.
If an exception occurs, insert nan values in the coordinates columns and print the word "nan inserted".
Please be patient and wait until the sentence 'OK. Finished' is printed


In [ ]:
j=0
for x in df['City']:  
        address = x
        try:         
            geolocator = Nominatim(user_agent="World_explorer")
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
            #df.loc[j,'lat']=latitude
            #df.loc[j,'lng']=longitude
            #print ('j=', j, 'city=' , df.loc[j,'City'], 'lat=', df.loc[j,'lat'], 'long =', df.loc[j,'lng'])
            
            #print ('j=', j)
            #print ('address=', x)
            
            #print (df.loc[j,'latdf['CityCoordinates'] = df['city'].apply(geolocator.geocode, timeout=1000000).apply(lambda x: eval_results(x))
            #print (df.loc[j,'lng'])
            #print('The geograpical coordinate of  Cities are {}, {}.'.format(latitude, longitude))
            #j=j+1
            # print ('j=', j)
        except:
            #print ('exception',address)
            df.loc[j,'lat']='nan'
            df.loc[j,'lng']='nan'
            #print ('j=', j, 'city=' , df.loc[j,'City'], 'lat=', df.loc[j,'lat'], 'long =', df.loc[j,'lng'])
            print ('nan inserted')
            j=j+1
            continue
        df.loc[j,'lat']=latitude
        df.loc[j,'lng']=longitude
        print ('j=', j, 'city=' , df.loc[j,'City'], 'lat=', df.loc[j,'lat'], 'long =', df.loc[j,'lng'])
        j=j+1
print ('OK. Finished')   

In [ ]:
df.head()


Drop rows with nan values in latitude or longitude fields (if any)

In [ ]:

#df.dropna(subset = ["lat"], inplace=True)
#df.dropna(inplace=True)
#df.info()
#df2= df.dropna(how='any',inplace=True) 
#df.dropna(axis=0, subset=['lat'],inplace=True)
df = df[df.lat != 'nan']
df = df[df.lat != 'nan']

In [ ]:
df.info() 

Reset df index in case nan rows were dropped

In [ ]:
df.reset_index(drop=True,inplace=True )

In [ ]:
df

In [ ]:
df.info()

# Create a World Map with all Countries' Capital Cities superimposed on top.

In [ ]:
# create map of World counteries using latitude and longitude values

map_World = folium.Map(location=[latitude, longitude], zoom_start=1.9)

# add markers to map
for lat, lng, label in zip(df['lat'], df['lng'], df['City']):
    try:
        
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=1,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_World) 
    except:
        pass


map_World

# Exploring  the World Capital Cities 

Next, we are going to start utilizing the Foursquare API to explore the World capital cities and segment them.

Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'JSTSDMFW35QUZS4STIZJVDT0ZJ20MEVH3INEGWEJOL2FEKOF' # your Foursquare ID
CLIENT_SECRET = 'FDUOZO42D4FJWCTHTBGHTP5DV0UAUNVXXZCWBRBZJALD0VLW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Let's explore the second country in our dataframe.

Get the country's  capital city name

In [ ]:
df.loc[2, 'City']

Get the Country's capital latitude and longitude values.

In [ ]:
neighborhood_latitude = df.loc[2, 'lat'] # neighborhood latitude value
neighborhood_longitude = df.loc[2, 'lng'] # neighborhood longitude value

neighborhood_name = df.loc[2, 'City'] # neighborhood name


print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

###### Now, let's get the top 100 venues that are in Abu Dhabi  within a radius of 6 km.

First, let's create the GET request URL. Name your URL url.

In [ ]:
limit = 100
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 60000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

Send the GET request and examine the resutls

In [ ]:
results = requests.get(url).json()
results

Build a function that extracts the category of the venue

In [ ]:
# function that extracts the category of the venue

def get_category_type(row):

    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe.

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

Let's create a function to repeat the same process to all the country capitals in the world

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=60000):     
    LIMIT =100
    limit= 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
             CLIENT_SECRET, 
             VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        try:
            # make the GET request      
            results = requests.get(url).json()["response"]['groups'][0]['items']
            # return only relevant information for each nearby venue
        except:
            print ('exception')
            continue
        venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        #except:
           # print ('exception')
           # continue
            
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['World Capital', 
                        'Latitude', 
                        'Longitude', 
                        'Venue', 
                        'Venue Latitude', 
                        'Venue Longitude', 
                        'Venue Category']    
            
    print ('finished')
    return(nearby_venues) 
    

Run the above function on each capital and create a new dataframe called World_venues.

In [ ]:
print ('please wait until a Finish message is displayed')
World_venues = getNearbyVenues(names=df['City'],
                                   latitudes=df['lat'],
                                   longitudes=df['lng']
                                  )
print ('Finished')
World_venues

# Letus check for douplicates

In [ ]:
duplicateDFRow = World_venues[World_venues.duplicated()]
df_duplicateDFRow =pd.DataFrame(duplicateDFRow)
df_duplicateDFRow
#print(duplicateDFRow)

Let's check the size and contents of the resulting dataframe

In [ ]:
print(World_venues.shape)
World_venues

In [ ]:
World_venues.info()

Letus drop duplicate venues

In [ ]:
World_venues = World_venues.drop_duplicates()

Let us check the size after dropping duplicates

In [ ]:
World_venues.info()

Let's check how many venues were returned for each country capital in the world

In [ ]:
World_venues.groupby('World Capital').count()

In [ ]:

dfa = pd.DataFrame(World_venues.groupby('World Capital').count())
#dfa
dfa.reindex()
#dfa.plot.barh(x='World Capital', y='Venue', title="Total number of venues for each national capital city");

#plot.show(block=True);

In [ ]:
dfa.info()

In [ ]:
import pandas as pd

import matplotlib.pyplot as plot
plot.rcParams["figure.figsize"] = [5, 60]
dfa= dfa.reset_index()
dfa.plot.barh(x="World Capital" , y="Venue", rot=0, title="Total number of venues for each national capital city");

plot.show(block=True);

In [ ]:

print( dfa.describe())

In [ ]:

dfa.plot.hist( by='Venue', bins=200,log=True, figsize = (15,5) )

In [ ]:
dfa.info()

In [ ]:
dfb=dfa['Venue']

In [ ]:
dfb.plot.hist( by='Venue', bins=50,log=True, figsize = (15,6) )

Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(World_venues['Venue Category'].unique())))


Let Us display the  uniques categorie 

In [ ]:

from pandas import DataFrame
uniques_categories= DataFrame (World_venues['Venue Category'].unique())
#uniques_categories
uniques_categories.sort_values(by=[0])
uniques_categories.reindex() 
uniques_categories.head()

Let us calculate the total number of venues in each category

In [ ]:
Venues_category_Statistics= DataFrame (World_venues['Venue Category'].value_counts())

#Venues_category_Statistics.reset_index()
#gapminder.columns = ['country','year','population','continent','life_exp','gdp_per_cap']
#Venues_category_Statistics.reset_index() 
#Venues_category_Statistics.reindex()

#Venues_category_Statistics.reset_index()
#Venues_category_Statistics.reindex()

In [ ]:
#Venues_category_Statistics_a = Venues_category_Statistics.reset_index()
Venues_category_Statistics_a = Venues_category_Statistics 

#Venues_category_Statistics_a.reindex()
Venues_category_Statistics_a.reset_index(inplace = True)
Venues_category_Statistics_a 

Rename columns

In [ ]:
Venues_category_Statistics_b=Venues_category_Statistics_a.rename(columns={'index':'Venue Category','Venue Category':'Count_Category' })
                 
Venues_category_Statistics_b

select needed rows only

In [ ]:
Venues_category_Statistics_c= Venues_category_Statistics_b.loc[(Venues_category_Statistics_b['Venue Category'] == 'Hotel') | (Venues_category_Statistics_b['Venue Category'] == 'Coffee Shop')| (Venues_category_Statistics_b['Venue Category'] == 'Restaurant')]
Venues_category_Statistics_c

In [ ]:
ax = Venues_category_Statistics_c.plot.bar(x='Venue Category', y='Count_Category', rot=0,figsize = (8,4),fontsize =14)
ax.set_ylabel('Count_Category')
from decimal import Decimal
import matplotlib.pyplot as plt
for p in ax.patches:
    ax.annotate('{:.5}'.format(Decimal(str(p.get_height()))), (p.get_x(), p.get_height()))
plt.tight_layout()
plt.show()


In [ ]:
#Venues_category_Statistics_c.info()
ss=Venues_category_Statistics_c.sum()
print ( ss)

Let us exclude all venues except Hotels, Coffee Shops and Restaurants 

In [ ]:
#indexNames = World_venues [ (World_venues ['Venue Category']  !=  ‘Hotel’)   ].index
#World_venues.drop(indexNames , inplace=False)
#World_venues [ World_venues ['Venue Category'].isin ('Hotel','Coffee Shop' )] 
tt = ['Hotel' , 'Coffee Shop','Restaurant' ]
World_venues1= World_venues[World_venues['Venue Category'].isin(tt)]
#reset_index(drop=True,inplace=True )
World_venues1.head(30)

Letus reset the index of the data frame

In [ ]:
World_venues1.reset_index(drop=True,inplace=True )
World_venues1.head(20)

In [ ]:
World_venues1.info()

In [ ]:
World_venues1=World_venues1.drop_duplicates()
World_venues1.info()

In [ ]:
World_venues=World_venues1
World_venues.head(10)

# Analyze Each World Country Capital

In [ ]:
World_venues

Do  one hot encoding

In [ ]:
# one hot encoding
World_onehot = pd.get_dummies(World_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
World_onehot['World Capital'] = World_venues['World Capital'] 

# move neighborhood column to the first column
fixed_columns = [World_onehot.columns[-1]] + list(World_onehot.columns[:-1])
World_onehot = World_onehot[fixed_columns]

World_onehot.head(30)


In [ ]:
World_onehot.info()

Next, let's group rows by World Capital Cities and by taking the mean of the frequency of occurrence of each category

In [ ]:
World_grouped = World_onehot.groupby('World Capital').mean().reset_index()
World_grouped.head()

Let's print each capital city  along with the  venues of interest 

In [ ]:
num_top_venues = 3

for hood in World_grouped['World Capital']:
    print("----"+hood+"----")
    temp = World_grouped[World_grouped['World Capital'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Let's put that into a pandas dataframe

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['World Capital']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
World_Capital_venues_sorted = pd.DataFrame(columns=columns)
World_Capital_venues_sorted['World Capital'] = World_grouped['World Capital']

for ind in np.arange(World_grouped.shape[0]):
    World_Capital_venues_sorted.iloc[ind, 1:] = return_most_common_venues(World_grouped.iloc[ind, :], num_top_venues)

World_Capital_venues_sorted 

In [ ]:
World_Capital_venues_sorted =World_Capital_venues_sorted .drop_duplicates()
World_Capital_venues_sorted.info()

# Cluster World Capital Cities

### Run k-means to cluster the world capital cities into 3 clusters.

In [ ]:
World_grouped


In [ ]:
World_grouped.info()

In [ ]:
# set number of clusters
kclusters = 3

World_grouped_clustering = World_grouped.drop('World Capital', 1)

# run k-means clustering
kmeans = KMeans(algorithm = 'full', init="k-means++",n_init=1000, max_iter=600, n_clusters=kclusters, random_state= 42).fit(World_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_ 
labels = kmeans.labels_
print(labels)
#World_grouped_clustering 

In [ ]:
labels1 = kmeans.labels_
labels1

In [ ]:
World_grouped["Clus_km"] = labels
World_grouped.head(20)

In [ ]:
World_grouped_clustering.loc [:, 'Hotel']

We can easily check the centroid values by averaging the features in each cluster

In [ ]:
World_grouped.groupby('Clus_km').mean()

In [ ]:
World_grouped.groupby('Clus_km').count()

Now, lets look at the distribution of market segment based on the number of Hptels, Coffeeshops and Restaurants

In [ ]:
X = World_grouped_clustering.values[:,0:]
X

In [ ]:
np.shape(X)

Diisribution of cities based on the frequency of the hotels and coffeshop venues

In [ ]:
fig = plt.figure(1, figsize=(9, 9))
area = 250* np.pi * ( X[:, 2])**2
plt.scatter(X[:, 0], X[:, 1], s=area, c=  labels1.astype(np.float), alpha= .5,  cmap ="rainbow", edgecolor='black')
plt.xlabel('Coffee Shop', fontsize=20)
plt.ylabel('Hotel', fontsize=20)

plt.show()

In [ ]:
fig = plt.figure(1, figsize=(9, 9))
area = 250* np.pi * ( X[:, 0])**2
plt.scatter(X[:, 1], X[:, 2], s=area, c=  labels1.astype(np.float), alpha= .5,  cmap ="rainbow", edgecolor='black')
plt.xlabel('Hotel', fontsize=20)
plt.ylabel('Restaurant', fontsize=20)

plt.show()

In [ ]:
ig = plt.figure(1, figsize=(9, 9))
area = 250* np.pi * ( X[:, 1])**2
plt.scatter(X[:, 0], X[:, 2], s=area, c=  labels1.astype(np.float), alpha= .5, cmap=plt.cm.rainbow, edgecolor='black')
plt.xlabel('Coffee SHop', fontsize=20)
plt.ylabel('Restaurant', fontsize=20)

plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 
fig = plt.figure(1, figsize=(10, 10))

plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

plt.cla()
#plt.fontsize = 25
ax.zaxis.set_tick_params(labelsize=14)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.zaxis.set_tick_params(grid_label =16)
plt.xlabel('Coffee Shop', fontsize=20)
plt.ylabel('Hotel', fontsize=20)
#plt.zlabel('Restaurant', fontsize=20)

ax.set_zlabel('Restaurant', fontsize=20)
ax.clabel( inline=True, fontsize=30)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c= labels1 .astype(np.float),s=250, alpha =.3, cmap=plt.cm.rainbow,edgecolor='black' )


Let us filter the df to get the cities that have venue data onlt 

In [ ]:
#dfa = df_A.drop_duplicates(subset=['my_icon_number'])
#dfb = df_B.drop_duplicates(subset=['my_icon_number'])
dfa = dfa.drop_duplicates(subset=['World Capital'])
#dfa

In [ ]:
#World_Capital_merged = World_Capital_merged.join(World_Capital_venues_sorted.set_index('World Capital'), on='City')
#tt =pd.DataFrame(dfa)
#df = df.join(tt.set_index('World Capital'), on='City')

dfb = pd.merge(dfa, df, left_on='World Capital',right_on='City',suffixes=(' ', ' '),  how='inner')

dfb

In [ ]:
dfb=dfb.drop_duplicates()
dfb.info()

### Let's create a new dataframe that includes the cluster as well as the  3 venues for of interest the element world capital cities.

In [ ]:
# add clustering labels
World_Capital_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)

World_Capital_merged = dfb

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood'CommonName'
World_Capital_merged = World_Capital_merged.join(World_Capital_venues_sorted.set_index('World Capital'), on='City')

World_Capital_merged  # che#ck the last columns!
#World_Capital_venues_sorted
#World_Capital_merged 


Drop the nan value from cluster lables

In [ ]:
World_Capital_merged.dropna (subset=['Cluster Labels'], inplace=True)
World_Capital_merged.head(300) 

In [ ]:
World_Capital_merged.info()

In [ ]:
World_Capital_merged=World_Capital_merged.drop_duplicates()
World_Capital_merged

# visualizing the resulting clusters

Drop nan values from cluster lables

In [ ]:
#World_Capital_merged.dropna( inplace=True)
#World_Capital_merged

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(World_Capital_merged['lat'], World_Capital_merged['lng'], World_Capital_merged['City'], World_Capital_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
   
    folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color=rainbow[int(cluster-1)],
        #olor='blue',
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        #fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

# To visualize  the map please copy the Giyhub-link of this notebook and past it in the following website:
https://nbviewer.jupyter.org/

Let up print some statistics

In [ ]:
World_onehot.head()

In [ ]:
World_onehot_grouped = World_onehot.groupby(by="World Capital").agg({'Coffee Shop': 'sum','Hotel': 'sum','Restaurant': 'sum'})
World_onehot_grouped =pd.DataFrame(World_onehot_grouped).reset_index() 
World_onehot_grouped

Calculate total target venues in each capital city

In [ ]:
World_onehot_grouped1 = World_onehot_grouped ["Coffee Shop"] + World_onehot_grouped["Hotel"]+ World_onehot_grouped["Restaurant"]
World_onehot_grouped1
World_onehot_grouped['total']=World_onehot_grouped1
World_onehot_grouped

Include other details of each capital city

In [ ]:
World_onehot_grouped_merged = pd.merge(World_onehot_grouped, World_Capital_merged, left_on='World Capital', right_on ='City')
#df_merge_difkey = pd.merge(df_row, df3, left_on='id', right_on='id')
World_onehot_grouped_merged.head(30)

In [ ]:
#World_venues = World_venues.drop_duplicates()
World_onehot_grouped_merged=World_onehot_grouped_merged.drop_duplicates()
World_onehot_grouped_merged.info()

Drow a histogram that depicts the distribution of features oneach city

In [ ]:
World_onehot_grouped_merged_W = World_onehot_grouped_merged [['World Capital_x', 'Hotel', 'Coffee Shop' , 'Restaurant' , 'total']] 
World_onehot_grouped_merged_W.hist( bins=25,log=False, figsize = (15,8) )


Drow a barchart that depicts the distribution of features oneach city

In [ ]:
import pandas as pd
import matplotlib.pyplot as plot

plot.rcParams["figure.figsize"] = [10, 300]
#dfa= dfa.reset_index()
World_onehot_grouped_merged_W = World_onehot_grouped_merged [['World Capital_x', 'Hotel', 'Coffee Shop' , 'Restaurant' , 'total']] 
World_onehot_grouped_merged_W.plot.barh(x="World Capital_x" ,  rot=0, title="Number of Selected Features' Venues in each national capital city");
plot.grid()
#plot.show(block=True);



In [ ]:
World_onehot_grouped_merged.info()

Let us drow heat map for total venues Worldwide


In [ ]:
World_onehot_grouped_merged['marker_color'] = pd.cut(World_onehot_grouped_merged['total'], bins=3, 
                              labels=[ ' #FFA500','#1E90FF', 'red' ])
m = folium.Map(location=[latitude, longitude], zoom_start=2)
#m = folium.Map(location=[39.2904, -76.6122],zoom_start=12)
for index, row in World_onehot_grouped_merged.iterrows():
    #label = folium.Popup(str(poi) + ' Cluster ' + str(World_onehot_grouped_merged.loc[index,'total']), parse_html=True)
    label = str(World_onehot_grouped_merged.loc[index,'City']) +',Venues= ' +str( World_onehot_grouped_merged.loc[index,'total'])
    folium.CircleMarker([row['lat'], row['lng']],
                    radius=3, popup= label, color=row['marker_color']).add_to(m)
m
    #print(World_onehot_grouped_merged)
   

Let us drow heat map for total hotels Worldwide


In [ ]:
World_onehot_grouped_merged['marker_color'] = pd.cut(World_onehot_grouped_merged['Hotel'], bins=3, 
                              labels=[ ' #FFA500','#1E90FF', 'red' ])
m = folium.Map(location=[latitude, longitude], zoom_start=2)
#m = folium.Map(location=[39.2904, -76.6122],zoom_start=12)
for index, row in World_onehot_grouped_merged.iterrows():
    #label = folium.Popup(str(poi) + ' Cluster ' + str(World_onehot_grouped_merged.loc[index,'total']), parse_html=True)
    label = str(World_onehot_grouped_merged.loc[index,'City']) +',Hotel= ' +str( World_onehot_grouped_merged.loc[index,'Hotel'])
    folium.CircleMarker([row['lat'], row['lng']],
                    radius=3, popup= label, color=row['marker_color']).add_to(m)
m
    #print(World_onehot_grouped_merged)
   

Let us drow heat map for Coffeeshops  Worldwide

In [ ]:
World_onehot_grouped_merged['marker_color'] = pd.cut(World_onehot_grouped_merged['Coffee Shop'], bins=3, 
                              labels=[ ' #FFA500','#1E90FF', 'red' ])
m = folium.Map(location=[latitude, longitude], zoom_start=2)
#m = folium.Map(location=[39.2904, -76.6122],zoom_start=12)
for index, row in World_onehot_grouped_merged.iterrows():
    #label = folium.Popup(str(poi) + ' Cluster ' + str(World_onehot_grouped_merged.loc[index,'total']), parse_html=True)
    label = str(World_onehot_grouped_merged.loc[index,'City']) +',Coffee Shop= ' +str( World_onehot_grouped_merged.loc[index,'Coffee Shop'])
    folium.CircleMarker([row['lat'], row['lng']],
                    radius=3, popup= label, color=row['marker_color']).add_to(m)
m
    #print(World_onehot_grouped_merged)

Detais of Cluster zero

Let us drow heat map for Resrurants  Worldwide

In [ ]:
World_onehot_grouped_merged.loc [World_onehot_grouped_merged['World Capital_x']== 'Lisbon']
#df.loc[df['column_name'] == some_value]

In [ ]:
World_onehot_grouped_merged['marker_color'] = pd.cut(World_onehot_grouped_merged['Restaurant'], bins=3, 
                              labels=[ ' #FFA500','#1E90FF', 'red' ])
m = folium.Map(location=[latitude, longitude], zoom_start=2)
#m = folium.Map(location=[39.2904, -76.6122],zoom_start=12)
for index, row in World_onehot_grouped_merged.iterrows():
    #label = folium.Popup(str(poi) + ' Cluster ' + str(World_onehot_grouped_merged.loc[index,'total']), parse_html=True)
    label = str(World_onehot_grouped_merged.loc[index,'City']) +',Restaurant= ' + str( World_onehot_grouped_merged.loc[index,'Restaurant'])
    folium.CircleMarker([row['lat'], row['lng']],
                    radius=3, popup= label, color=row['marker_color']).add_to(m)
m
    #print(World_onehot_grouped_merged)

In [ ]:
Cluster_zero =  World_onehot_grouped_merged[World_onehot_grouped_merged['Cluster Labels']== 0.0 ]

Cluster_zero.sort_values(by=['total'], inplace=True, ascending=False)
Cluster_zero.reset_index(drop=True,inplace=True )
Cluster_zero.head()

In [ ]:
Cluster_zero[['World Capital_x','Country','Coffee Shop', 'Hotel', 'Restaurant','total','Venue','Cluster Labels','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']]

Features of Clster zero

In [ ]:
# Draw a vertical bar chart

#Cluster_zero.plot.bar(x="City", y="Visits", rot=70, title="Number of tourist visits - Year 2018");

#plot.show(block=True);
World_onehot_grouped_merged.info()

In [ ]:
Cluster_one =  World_onehot_grouped_merged[World_onehot_grouped_merged['Cluster Labels']== 1.0 ]
Cluster_one.sort_values(by=['total'], inplace=True, ascending=False)
Cluster_one.reset_index(drop=True,inplace=True )
Cluster_one

In [ ]:
Cluster_one[['World Capital_x','Country','Coffee Shop', 'Hotel', 'Restaurant','total','Venue','Cluster Labels','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']]

In [ ]:
Cluster_tow =  World_onehot_grouped_merged[World_onehot_grouped_merged['Cluster Labels']== 2.0 ]
Cluster_tow.sort_values(by=['total'], inplace=True, ascending=False)
Cluster_tow.reset_index(drop=True,inplace=True )
Cluster_tow

In [ ]:
Cluster_tow[['World Capital_x','Country','Coffee Shop', 'Hotel', 'Restaurant','total','Venue','Cluster Labels','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']]

Depict Clusters' Features

In [ ]:
World_onehot_grouped_merged.head()

In [ ]:
World_onehot_grouped_merged_clusters=World_onehot_grouped_merged

World_onehot_grouped_merged_clusters = World_onehot_grouped_merged_clusters.groupby("Cluster Labels").agg ({'Coffee Shop': 'sum','Hotel': 'sum','Restaurant': 'sum', 'total': 'sum'})

World_onehot_grouped_merged_clusters [['Coffee Shop', 'Hotel','Restaurant']]

In [ ]:
World_onehot_grouped_merged_clusters1=World_onehot_grouped_merged_clusters

In [ ]:
# Draw  pie  charts

#World_onehot_grouped_merged_clusters.plot.bar(x='Cluster Labels', y=['Coffee Shop',]'Coffee Shop''Hotel, rot=70, title="Number of venues per cluster");

World_onehot_grouped_merged_clusters.plot.pie(subplots=True, figsize=(15, 30),shadow = True,fontsize=12, autopct='%1.2f%%', startangle=0,legend=False )


In [ ]:

World_onehot_grouped_merged_clusters [['Hotel','Coffee Shop', 'Restaurant']]

In [ ]:
ax = World_onehot_grouped_merged_clusters.plot.bar( rot=0,figsize = (6,3),fontsize =14)
ax.set_ylabel('Count_Category')
from decimal import Decimal
import matplotlib.pyplot as plt
for p in ax.patches:
    ax.annotate('{:.5}'.format(Decimal(str(p.get_height()))), (p.get_x(), p.get_height()))
plt.tight_layout()
plt.show()

# Hierarchical Agglomerative Clustering   

In [ ]:
import numpy as np 
import pandas as pd
from scipy import ndimage 
from scipy.cluster import hierarchy 
from scipy.spatial import distance_matrix 
from matplotlib import pyplot as plt 
from sklearn import manifold, datasets 
from sklearn.cluster import AgglomerativeClustering 
from sklearn.datasets.samples_generator import make_blobs 
%matplotlib inline
print ('ok')

In [ ]:
agglom = AgglomerativeClustering(n_clusters = 3, linkage = 'single')

In [ ]:
agglom.fit(World_grouped_clustering)

In [ ]:
World_grouped.head(5)

In [ ]:
World_grouped_clustering.head()

Dendrogram Associated for the Agglomerative Hierarchical Clustering

In [ ]:
dist_matrix = distance_matrix(World_grouped[['Coffee Shop', 'Hotel', 'Restaurant']],World_grouped[['Coffee Shop', 'Hotel', 'Restaurant']]) 
print(dist_matrix)

Using the linkage class from hierarchy, pass in the parameters:

The distance matrix
'complete' for complete linkage

Save the result to a variable called Z

In [ ]:
import pylab
import scipy.cluster.hierarchy
Z = hierarchy.linkage(dist_matrix, 'average')

A Hierarchical clustering is typically visualized as a dendrogram as shown in the following cell. Each merge is represented by a horizontal line. The y-coordinate of the horizontal line is the similarity of the two clusters that were merged, where cities are viewed as singleton clusters. 
By moving up from the bottom layer to the top node, a dendrogram allows us to reconstruct the history of merges that resulted in the depicted clustering. 

Next, we will save the dendrogram to a variable called <b>dendro</b>. In doing this, the dendrogram will also be displayed.
Using the <b> dendrogram </b> class from hierarchy, pass in the parameter:
<ul> <li> Z </li> </ul>

In [ ]:
fig = pylab.figure(figsize=(18,80))
def llf(id):
    return '[%s ]' % (World_grouped['World Capital'][id],) 
    
dendro = hierarchy.dendrogram(Z,  leaf_label_func=llf, leaf_rotation=0, leaf_font_size =12, orientation = 'right')

#dendro = hierarchy.dendrogram(Z)
#fig = pylab.figure(figsize=(8,60))
#dendro = hierarchy.dendrogram(Z,  leaf_rotation=0, leaf_font_size =12, orientation = 'right')

In [ ]:
label11= agglom.labels_


In [ ]:
np.info(label11)

In [ ]:
World_grouped['Clus_km'].value= ['label11']
World_grouped

# End of Capstone Assignment